In [3]:
import os
from _src import LOG_DIR, DB_DIR
from _src import search_log_files

In [5]:
import os
import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Function to convert hex to rgba
def hex_to_rgba(hex_color, alpha=0.2):
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, {alpha})'

# Plot and save figure function
def plot_and_save_fig(sampler_files, db_folder, color_scale, save_path, width=800, height=600):
    fig = go.Figure()

    # Loop over each sampler and assign colors from the Plotly qualitative scale
    for idx, (sampler, db_files) in enumerate(sampler_files.items()):
        dataframes = []
        for db_file in db_files:
            path = os.path.join(db_folder, db_file)
            storage_url = f"sqlite:///{path}"

            study_name = db_file.split('_bo_')[1].replace('.db', '')
            study_name = "bo_" + study_name if "parafac" in sampler else "ackley_bo_" + study_name

            study = optuna.load_study(study_name=study_name, storage=storage_url)

            df = study.trials_dataframe().copy()
            if study.direction == optuna.study.StudyDirection.MAXIMIZE:
                df['best_value'] = df['value'].cummax()
            else:
                df['best_value'] = df['value'].cummin()
            dataframes.append(df[['number', 'best_value']])

        merged_df = pd.concat(dataframes, axis=1, keys=[f'seed_{i}' for i in range(len(db_files))])
        best_values = merged_df.xs('best_value', axis=1, level=1)
        mean_best = best_values.mean(axis=1)
        std_best = best_values.std(axis=1)

        upper_bound = mean_best + std_best
        lower_bound = mean_best - std_best

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=mean_best,
            mode='lines',
            name=f'Mean Best Value ({sampler})',
            line=dict(color=color_scale[idx])
        ))

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=upper_bound,
            mode='lines',
            line=dict(width=0),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=lower_bound,
            mode='lines',
            line=dict(width=0),
            fill='tonexty',
            fillcolor=hex_to_rgba(color_scale[idx]),
            showlegend=False
        ))

    fig.update_layout(
        title="Best Value with Shaded Error Region for Different Samplers",
        xaxis_title="Iteration",
        yaxis_title="Best Value",
        template="plotly_white",
        legend=dict(
            orientation="h",
            x=0,
            y=1
        ),
        width=width,
        height=height
    )

    fig.write_image(save_path)


if __name__ == '__main__':
    result_path = "/Users/keisukeonoue/ws/constrained_BO_v2/results_ackley_5"
    db_folder = os.path.join(result_path, "dbs")

    dir_path = DB_DIR

    problem_setting = {
        "dim": ["dim2", "dim3", "dim5", "dim7"],
        "model": ["random", "gp", "tpe", "parafac"]
    }

    parafac_setting = {
        "rank": ["rank1", "rank2", "rank3", "rank4"],
        "mask": ["mask0.3"],
        "tradeoff": ["tradeoff3.0"]
    }

    log_files = {dim: {} for dim in problem_setting["dim"]}



    for dim in problem_setting["dim"]:
        for rank in parafac_setting["rank"]:
            for tradeoff in parafac_setting["tradeoff"]:
                sampler_files = {sampler: [] for sampler in problem_setting["model"] if sampler != "parafac"}

                for model in problem_setting["model"]:
                    if model == "parafac":
                        for mask in parafac_setting["mask"]:
                            parafac_name = f"{model}_rank{rank}_mask{mask}_tradeoff{tradeoff}"
                            sampler_files[parafac_name] = []

                            files = search_log_files(dir_path, [dim, rank, model, mask, tradeoff])
                            if len(files) == 5:
                                sampler_files[parafac_name].extend(files)
                    else:
                        files = search_log_files(dir_path, [dim, model])
                        if len(files) == 5:
                            sampler_files[model].extend(files)

                log_files[dim] = sampler_files

                color_scale = px.colors.qualitative.Plotly + px.colors.qualitative.D3 + px.colors.qualitative.G10 + px.colors.qualitative.T10
                save_path = os.path.join(result_path, "images", f"best_values_plot_{dim}_{rank}_{tradeoff}.png")
                
                plot_and_save_fig(
                    sampler_files, 
                    db_folder, 
                    color_scale, 
                    save_path,
                    width=1600,
                    height=1200)


ValueError: No objects to concatenate